In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

--- LogisticRegression ---

In [7]:
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

selector= RFE(estimator= lr, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.45      0.52      1125
           1       0.35      0.53      0.42       628

    accuracy                           0.48      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.48      0.49      1753

Precision: 0.3487394957983193
Accuracy: 0.47746719908727897
AUC: 0.4981302193913658


--- DecisionTreeClassifier ---

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

selector= RFE(estimator= dt, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.64      0.41      0.50      1125
           1       0.35      0.57      0.44       628

    accuracy                           0.47      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.47      0.48      1753

Precision: 0.3535749265426053
Accuracy: 0.47119224187107817
AUC: 0.49408704883227184


--- Tuned - DecisionTreeClassifier ---

In [9]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.6007491644129928


In [10]:
tuned_dt= gs_dt.best_estimator_

selector= RFE(estimator= tuned_dt, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.67      0.27      0.38      1125
           1       0.37      0.77      0.50       628

    accuracy                           0.45      1753
   macro avg       0.52      0.52      0.44      1753
weighted avg       0.56      0.45      0.42      1753

Precision: 0.36906584992343033
Accuracy: 0.4466628636622932
AUC: 0.515491153573956


--- Random Forest ---

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)

selector= RFE(estimator= rf, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [12]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.26      0.37      1125
           1       0.36      0.73      0.48       628

    accuracy                           0.43      1753
   macro avg       0.50      0.50      0.43      1753
weighted avg       0.54      0.43      0.41      1753

Precision: 0.3573643410852713
Accuracy: 0.4318311466058186
AUC: 0.5039299363057326


In [13]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best Precision Score: 0.6475732828947862


In [14]:
tuned_rf= gs_tuned_rf.best_estimator_

selector= RFE(estimator= tuned_rf, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.67      0.23      0.34      1125
           1       0.37      0.80      0.50       628

    accuracy                           0.43      1753
   macro avg       0.52      0.51      0.42      1753
weighted avg       0.56      0.43      0.40      1753

Precision: 0.3651603498542274
Accuracy: 0.4306902452937821
AUC: 0.5107728237791932


--- AdaBoost ---

In [15]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)

selector= RFE(estimator= ada, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [16]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.63      0.23      0.34      1125
           1       0.35      0.75      0.48       628

    accuracy                           0.42      1753
   macro avg       0.49      0.49      0.41      1753
weighted avg       0.53      0.42      0.39      1753

Precision: 0.3545727136431784
Accuracy: 0.4204221334854535
AUC: 0.49921868365180466


In [17]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6501382975903532


In [18]:
tuned_ada= gs_ada.best_estimator_

selector= RFE(estimator= tuned_ada, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.62      0.13      0.21      1125
           1       0.36      0.86      0.50       628

    accuracy                           0.39      1753
   macro avg       0.49      0.49      0.36      1753
weighted avg       0.53      0.39      0.32      1753

Precision: 0.3554533508541393
Accuracy: 0.3907586993725043
AUC: 0.46966878980891724


--- GradientBoosting ---

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)

selector= RFE(estimator= grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.67      0.13      0.22      1125
           1       0.36      0.88      0.51       628

    accuracy                           0.40      1753
   macro avg       0.52      0.51      0.37      1753
weighted avg       0.56      0.40      0.32      1753

Precision: 0.36203522504892366
Accuracy: 0.4004563605248146
AUC: 0.5033418259023354


In [20]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6765600147562179


In [21]:
tuned_grb= gs_grb.best_estimator_

selector= RFE(estimator= tuned_grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.69      0.17      0.27      1125
           1       0.37      0.87      0.52       628

    accuracy                           0.42      1753
   macro avg       0.53      0.52      0.39      1753
weighted avg       0.58      0.42      0.36      1753

Precision: 0.3680649526387009
Accuracy: 0.419281232173417
AUC: 0.5159688605803255


--- XGB ---

In [22]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)

selector= RFE(estimator= xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.67      0.25      0.36      1125
           1       0.37      0.77      0.50       628

    accuracy                           0.44      1753
   macro avg       0.52      0.51      0.43      1753
weighted avg       0.56      0.44      0.41      1753

Precision: 0.3656884875846501
Accuracy: 0.4381061038220194
AUC: 0.5164253361641896


In [23]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_

selector= RFE(estimator= tuned_xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.6739795194819881
              precision    recall  f1-score   support

           0       0.69      0.15      0.25      1125
           1       0.37      0.88      0.52       628

    accuracy                           0.41      1753
   macro avg       0.53      0.51      0.38      1753
weighted avg       0.57      0.41      0.34      1753

Precision: 0.36538461538461536
Accuracy: 0.41015402167712495
AUC: 0.5069072894550603


--- LGBM ---

In [24]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)

selector= RFE(estimator= lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2256
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002344 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2246
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [25]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_

selector= RFE(estimator= tuned_lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2256
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.6773939636916706
[LightGBM] [Warning] min_data_in_leaf is set=30, min_

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1964
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightG

--- CatBoost ---

In [26]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)

selector= RFE(estimator= cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.020521
0:	learn: 0.6872208	total: 145ms	remaining: 2m 25s
1:	learn: 0.6824308	total: 152ms	remaining: 1m 15s
2:	learn: 0.6784063	total: 159ms	remaining: 52.7s
3:	learn: 0.6746470	total: 164ms	remaining: 40.9s
4:	learn: 0.6719916	total: 169ms	remaining: 33.7s
5:	learn: 0.6684167	total: 174ms	remaining: 28.9s
6:	learn: 0.6653548	total: 179ms	remaining: 25.4s
7:	learn: 0.6622485	total: 185ms	remaining: 23s
8:	learn: 0.6591303	total: 191ms	remaining: 21s
9:	learn: 0.6566801	total: 196ms	remaining: 19.4s
10:	learn: 0.6525855	total: 202ms	remaining: 18.1s
11:	learn: 0.6499531	total: 207ms	remaining: 17s
12:	learn: 0.6480832	total: 214ms	remaining: 16.2s
13:	learn: 0.6436020	total: 220ms	remaining: 15.5s
14:	learn: 0.6404142	total: 226ms	remaining: 14.8s
15:	learn: 0.6381851	total: 232ms	remaining: 14.2s
16:	learn: 0.6368099	total: 236ms	remaining: 13.7s
17:	learn: 0.6341267	total: 242ms	remaining: 13.2s
18:	learn: 0.6326452	total: 248ms	remaining: 12.8s
19:	learn: 0.63

181:	learn: 0.5173023	total: 1.22s	remaining: 5.48s
182:	learn: 0.5169507	total: 1.23s	remaining: 5.47s
183:	learn: 0.5165982	total: 1.23s	remaining: 5.47s
184:	learn: 0.5161766	total: 1.24s	remaining: 5.46s
185:	learn: 0.5158846	total: 1.24s	remaining: 5.45s
186:	learn: 0.5150674	total: 1.25s	remaining: 5.43s
187:	learn: 0.5148201	total: 1.25s	remaining: 5.42s
188:	learn: 0.5144793	total: 1.26s	remaining: 5.41s
189:	learn: 0.5141883	total: 1.27s	remaining: 5.4s
190:	learn: 0.5136560	total: 1.27s	remaining: 5.39s
191:	learn: 0.5134703	total: 1.28s	remaining: 5.38s
192:	learn: 0.5131688	total: 1.28s	remaining: 5.37s
193:	learn: 0.5128693	total: 1.29s	remaining: 5.36s
194:	learn: 0.5125905	total: 1.29s	remaining: 5.34s
195:	learn: 0.5122844	total: 1.3s	remaining: 5.33s
196:	learn: 0.5119839	total: 1.3s	remaining: 5.32s
197:	learn: 0.5116404	total: 1.31s	remaining: 5.31s
198:	learn: 0.5113581	total: 1.32s	remaining: 5.3s
199:	learn: 0.5109679	total: 1.32s	remaining: 5.29s
200:	learn: 0.51

355:	learn: 0.4651279	total: 2.3s	remaining: 4.16s
356:	learn: 0.4648450	total: 2.31s	remaining: 4.16s
357:	learn: 0.4645343	total: 2.32s	remaining: 4.15s
358:	learn: 0.4642997	total: 2.33s	remaining: 4.16s
359:	learn: 0.4640350	total: 2.34s	remaining: 4.16s
360:	learn: 0.4637752	total: 2.35s	remaining: 4.17s
361:	learn: 0.4634600	total: 2.37s	remaining: 4.17s
362:	learn: 0.4631549	total: 2.37s	remaining: 4.17s
363:	learn: 0.4630020	total: 2.38s	remaining: 4.17s
364:	learn: 0.4627319	total: 2.4s	remaining: 4.17s
365:	learn: 0.4625107	total: 2.41s	remaining: 4.17s
366:	learn: 0.4622487	total: 2.42s	remaining: 4.17s
367:	learn: 0.4619903	total: 2.42s	remaining: 4.17s
368:	learn: 0.4617473	total: 2.44s	remaining: 4.17s
369:	learn: 0.4615884	total: 2.45s	remaining: 4.17s
370:	learn: 0.4612899	total: 2.46s	remaining: 4.17s
371:	learn: 0.4609604	total: 2.47s	remaining: 4.18s
372:	learn: 0.4606683	total: 2.49s	remaining: 4.18s
373:	learn: 0.4604629	total: 2.5s	remaining: 4.18s
374:	learn: 0.4

528:	learn: 0.4198335	total: 4.42s	remaining: 3.94s
529:	learn: 0.4195174	total: 4.44s	remaining: 3.93s
530:	learn: 0.4193132	total: 4.45s	remaining: 3.93s
531:	learn: 0.4190140	total: 4.47s	remaining: 3.93s
532:	learn: 0.4188009	total: 4.49s	remaining: 3.93s
533:	learn: 0.4185752	total: 4.51s	remaining: 3.93s
534:	learn: 0.4183763	total: 4.52s	remaining: 3.93s
535:	learn: 0.4180623	total: 4.54s	remaining: 3.93s
536:	learn: 0.4176292	total: 4.56s	remaining: 3.93s
537:	learn: 0.4174201	total: 4.58s	remaining: 3.93s
538:	learn: 0.4171924	total: 4.6s	remaining: 3.93s
539:	learn: 0.4168814	total: 4.61s	remaining: 3.93s
540:	learn: 0.4166475	total: 4.63s	remaining: 3.93s
541:	learn: 0.4163431	total: 4.65s	remaining: 3.93s
542:	learn: 0.4160873	total: 4.67s	remaining: 3.93s
543:	learn: 0.4158555	total: 4.68s	remaining: 3.92s
544:	learn: 0.4155674	total: 4.7s	remaining: 3.92s
545:	learn: 0.4153479	total: 4.71s	remaining: 3.92s
546:	learn: 0.4148384	total: 4.72s	remaining: 3.91s
547:	learn: 0.

695:	learn: 0.3775479	total: 6.32s	remaining: 2.76s
696:	learn: 0.3773565	total: 6.32s	remaining: 2.75s
697:	learn: 0.3771419	total: 6.33s	remaining: 2.74s
698:	learn: 0.3769056	total: 6.34s	remaining: 2.73s
699:	learn: 0.3766585	total: 6.35s	remaining: 2.72s
700:	learn: 0.3764516	total: 6.36s	remaining: 2.71s
701:	learn: 0.3762277	total: 6.37s	remaining: 2.7s
702:	learn: 0.3760114	total: 6.38s	remaining: 2.69s
703:	learn: 0.3757662	total: 6.39s	remaining: 2.69s
704:	learn: 0.3755456	total: 6.41s	remaining: 2.68s
705:	learn: 0.3753166	total: 6.42s	remaining: 2.67s
706:	learn: 0.3751551	total: 6.42s	remaining: 2.66s
707:	learn: 0.3749795	total: 6.43s	remaining: 2.65s
708:	learn: 0.3746897	total: 6.44s	remaining: 2.64s
709:	learn: 0.3744132	total: 6.45s	remaining: 2.63s
710:	learn: 0.3741705	total: 6.46s	remaining: 2.63s
711:	learn: 0.3738932	total: 6.47s	remaining: 2.62s
712:	learn: 0.3736613	total: 6.48s	remaining: 2.61s
713:	learn: 0.3733717	total: 6.49s	remaining: 2.6s
714:	learn: 0.

...

176:	learn: 0.5318428	total: 880ms	remaining: 4.09s
177:	learn: 0.5316308	total: 887ms	remaining: 4.09s
178:	learn: 0.5314975	total: 891ms	remaining: 4.09s
179:	learn: 0.5313239	total: 898ms	remaining: 4.09s
180:	learn: 0.5310445	total: 903ms	remaining: 4.09s
181:	learn: 0.5307719	total: 909ms	remaining: 4.08s
182:	learn: 0.5306515	total: 914ms	remaining: 4.08s
183:	learn: 0.5305150	total: 919ms	remaining: 4.08s
184:	learn: 0.5302948	total: 926ms	remaining: 4.08s
185:	learn: 0.5298800	total: 932ms	remaining: 4.08s
186:	learn: 0.5296237	total: 937ms	remaining: 4.07s
187:	learn: 0.5291400	total: 944ms	remaining: 4.08s
188:	learn: 0.5288333	total: 948ms	remaining: 4.07s
189:	learn: 0.5285551	total: 953ms	remaining: 4.06s
190:	learn: 0.5283200	total: 959ms	remaining: 4.06s
191:	learn: 0.5280453	total: 964ms	remaining: 4.05s
192:	learn: 0.5277318	total: 968ms	remaining: 4.05s
193:	learn: 0.5274772	total: 975ms	remaining: 4.05s
194:	learn: 0.5272211	total: 979ms	remaining: 4.04s
195:	learn: 

348:	learn: 0.4922148	total: 1.76s	remaining: 3.29s
349:	learn: 0.4919846	total: 1.77s	remaining: 3.29s
350:	learn: 0.4918345	total: 1.77s	remaining: 3.28s
351:	learn: 0.4916420	total: 1.78s	remaining: 3.28s
352:	learn: 0.4914748	total: 1.78s	remaining: 3.27s
353:	learn: 0.4912386	total: 1.79s	remaining: 3.27s
354:	learn: 0.4910582	total: 1.79s	remaining: 3.26s
355:	learn: 0.4909150	total: 1.8s	remaining: 3.25s
356:	learn: 0.4906881	total: 1.8s	remaining: 3.25s
357:	learn: 0.4905215	total: 1.81s	remaining: 3.25s
358:	learn: 0.4902850	total: 1.81s	remaining: 3.24s
359:	learn: 0.4901118	total: 1.82s	remaining: 3.23s
360:	learn: 0.4897920	total: 1.83s	remaining: 3.23s
361:	learn: 0.4896378	total: 1.83s	remaining: 3.23s
362:	learn: 0.4894965	total: 1.83s	remaining: 3.22s
363:	learn: 0.4893492	total: 1.84s	remaining: 3.22s
364:	learn: 0.4889634	total: 1.84s	remaining: 3.21s
365:	learn: 0.4887574	total: 1.85s	remaining: 3.2s
366:	learn: 0.4886104	total: 1.85s	remaining: 3.2s
367:	learn: 0.48

516:	learn: 0.4571711	total: 2.65s	remaining: 2.48s
517:	learn: 0.4569620	total: 2.66s	remaining: 2.47s
518:	learn: 0.4568083	total: 2.66s	remaining: 2.47s
519:	learn: 0.4565776	total: 2.67s	remaining: 2.46s
520:	learn: 0.4564330	total: 2.67s	remaining: 2.46s
521:	learn: 0.4561433	total: 2.68s	remaining: 2.45s
522:	learn: 0.4559597	total: 2.68s	remaining: 2.45s
523:	learn: 0.4557944	total: 2.69s	remaining: 2.44s
524:	learn: 0.4554770	total: 2.69s	remaining: 2.44s
525:	learn: 0.4553036	total: 2.7s	remaining: 2.43s
526:	learn: 0.4551083	total: 2.7s	remaining: 2.42s
527:	learn: 0.4548255	total: 2.71s	remaining: 2.42s
528:	learn: 0.4545958	total: 2.71s	remaining: 2.41s
529:	learn: 0.4544544	total: 2.72s	remaining: 2.41s
530:	learn: 0.4543055	total: 2.72s	remaining: 2.4s
531:	learn: 0.4541205	total: 2.73s	remaining: 2.4s
532:	learn: 0.4539026	total: 2.73s	remaining: 2.39s
533:	learn: 0.4536898	total: 2.74s	remaining: 2.39s
534:	learn: 0.4535420	total: 2.74s	remaining: 2.38s
535:	learn: 0.45

686:	learn: 0.4242008	total: 3.54s	remaining: 1.61s
687:	learn: 0.4239670	total: 3.54s	remaining: 1.61s
688:	learn: 0.4237391	total: 3.55s	remaining: 1.6s
689:	learn: 0.4235568	total: 3.55s	remaining: 1.6s
690:	learn: 0.4233192	total: 3.56s	remaining: 1.59s
691:	learn: 0.4231367	total: 3.56s	remaining: 1.58s
692:	learn: 0.4229655	total: 3.57s	remaining: 1.58s
693:	learn: 0.4227923	total: 3.57s	remaining: 1.57s
694:	learn: 0.4226130	total: 3.58s	remaining: 1.57s
695:	learn: 0.4224739	total: 3.58s	remaining: 1.56s
696:	learn: 0.4222999	total: 3.59s	remaining: 1.56s
697:	learn: 0.4221151	total: 3.59s	remaining: 1.55s
698:	learn: 0.4220155	total: 3.6s	remaining: 1.55s
699:	learn: 0.4218632	total: 3.6s	remaining: 1.54s
700:	learn: 0.4216852	total: 3.61s	remaining: 1.54s
701:	learn: 0.4215259	total: 3.61s	remaining: 1.53s
702:	learn: 0.4212688	total: 3.62s	remaining: 1.53s
703:	learn: 0.4210940	total: 3.62s	remaining: 1.52s
704:	learn: 0.4209774	total: 3.63s	remaining: 1.52s
705:	learn: 0.42

859:	learn: 0.3958794	total: 4.42s	remaining: 720ms
860:	learn: 0.3957471	total: 4.43s	remaining: 715ms
861:	learn: 0.3956094	total: 4.43s	remaining: 710ms
862:	learn: 0.3954412	total: 4.44s	remaining: 704ms
863:	learn: 0.3952045	total: 4.44s	remaining: 699ms
864:	learn: 0.3950358	total: 4.45s	remaining: 694ms
865:	learn: 0.3948858	total: 4.45s	remaining: 688ms
866:	learn: 0.3947337	total: 4.46s	remaining: 683ms
867:	learn: 0.3946054	total: 4.46s	remaining: 678ms
868:	learn: 0.3945188	total: 4.46s	remaining: 673ms
869:	learn: 0.3942388	total: 4.47s	remaining: 668ms
870:	learn: 0.3940582	total: 4.47s	remaining: 663ms
871:	learn: 0.3939524	total: 4.48s	remaining: 658ms
872:	learn: 0.3938383	total: 4.48s	remaining: 652ms
873:	learn: 0.3937023	total: 4.49s	remaining: 647ms
874:	learn: 0.3935582	total: 4.5s	remaining: 642ms
875:	learn: 0.3934042	total: 4.5s	remaining: 637ms
876:	learn: 0.3933205	total: 4.51s	remaining: 632ms
877:	learn: 0.3932117	total: 4.51s	remaining: 627ms
878:	learn: 0.

In [27]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6423855	total: 4.06ms	remaining: 402ms
1:	learn: 0.6106768	total: 8.52ms	remaining: 417ms
2:	learn: 0.5643117	total: 11.7ms	remaining: 379ms
3:	learn: 0.5612499	total: 16.2ms	remaining: 389ms
4:	learn: 0.5552844	total: 20.1ms	remaining: 381ms
5:	learn: 0.5525986	total: 24ms	remaining: 377ms
6:	learn: 0.5416322	total: 27.2ms	remaining: 361ms
7:	learn: 0.5389803	total: 30.8ms	remaining: 354ms
8:	learn: 0.5372275	total: 34ms	remaining: 343ms
9:	learn: 0.5342521	total: 36.9ms	remaining: 332ms
10:	learn: 0.5316575	total: 39.9ms	remaining: 323ms
11:	learn: 0.5261891	total: 43.1ms	remaining: 316ms
12:	learn: 0.5183570	total: 47.2ms	remaining: 316ms
13:	learn: 0.5159393	total: 52.7ms	remaining: 324ms
14:	learn: 0.5128910	total: 56ms	remaining: 317ms
15:	learn: 0.5090084	total: 59.3ms	remaining: 311ms
16:	learn: 0.5058439	total: 63.2ms	remaining: 308ms
17:	learn: 0.5008793	total: 66.4ms	remaining: 302ms
18:	learn: 0.49803

69:	learn: 0.3700226	total: 259ms	remaining: 111ms
70:	learn: 0.3680482	total: 263ms	remaining: 107ms
71:	learn: 0.3663321	total: 266ms	remaining: 104ms
72:	learn: 0.3643882	total: 270ms	remaining: 100ms
73:	learn: 0.3629486	total: 275ms	remaining: 96.7ms
74:	learn: 0.3620713	total: 279ms	remaining: 93ms
75:	learn: 0.3602447	total: 284ms	remaining: 89.6ms
76:	learn: 0.3590914	total: 287ms	remaining: 85.9ms
77:	learn: 0.3574481	total: 292ms	remaining: 82.3ms
78:	learn: 0.3556383	total: 295ms	remaining: 78.4ms
79:	learn: 0.3538713	total: 299ms	remaining: 74.8ms
80:	learn: 0.3521680	total: 303ms	remaining: 71ms
81:	learn: 0.3498601	total: 306ms	remaining: 67.3ms
82:	learn: 0.3481627	total: 311ms	remaining: 63.7ms
83:	learn: 0.3461229	total: 315ms	remaining: 59.9ms
84:	learn: 0.3447687	total: 318ms	remaining: 56.1ms
85:	learn: 0.3430704	total: 322ms	remaining: 52.4ms
86:	learn: 0.3416706	total: 326ms	remaining: 48.8ms
87:	learn: 0.3405180	total: 330ms	remaining: 45ms
88:	learn: 0.3395051	t

32:	learn: 0.4495323	total: 121ms	remaining: 246ms
33:	learn: 0.4475882	total: 125ms	remaining: 243ms
34:	learn: 0.4455201	total: 128ms	remaining: 239ms
35:	learn: 0.4434474	total: 133ms	remaining: 236ms
36:	learn: 0.4407479	total: 136ms	remaining: 232ms
37:	learn: 0.4380905	total: 140ms	remaining: 229ms
38:	learn: 0.4360898	total: 144ms	remaining: 226ms
39:	learn: 0.4335087	total: 148ms	remaining: 222ms
40:	learn: 0.4311366	total: 153ms	remaining: 220ms
41:	learn: 0.4292797	total: 157ms	remaining: 217ms
42:	learn: 0.4271721	total: 161ms	remaining: 213ms
43:	learn: 0.4256219	total: 165ms	remaining: 209ms
44:	learn: 0.4237330	total: 168ms	remaining: 205ms
45:	learn: 0.4214862	total: 173ms	remaining: 203ms
46:	learn: 0.4193317	total: 178ms	remaining: 201ms
47:	learn: 0.4172243	total: 182ms	remaining: 197ms
48:	learn: 0.4152107	total: 186ms	remaining: 194ms
49:	learn: 0.4132418	total: 191ms	remaining: 191ms
50:	learn: 0.4118320	total: 196ms	remaining: 189ms
51:	learn: 0.4104542	total: 201

93:	learn: 0.3816063	total: 200ms	remaining: 12.8ms
94:	learn: 0.3788038	total: 203ms	remaining: 10.7ms
95:	learn: 0.3766931	total: 206ms	remaining: 8.57ms
96:	learn: 0.3755837	total: 208ms	remaining: 6.42ms
97:	learn: 0.3740007	total: 210ms	remaining: 4.28ms
98:	learn: 0.3722818	total: 211ms	remaining: 2.13ms
99:	learn: 0.3709204	total: 213ms	remaining: 0us
0:	learn: 0.6463174	total: 1.8ms	remaining: 178ms
1:	learn: 0.6104666	total: 4.65ms	remaining: 228ms
2:	learn: 0.6018633	total: 6.9ms	remaining: 223ms
3:	learn: 0.5954534	total: 9.08ms	remaining: 218ms
4:	learn: 0.5707936	total: 11.7ms	remaining: 221ms
5:	learn: 0.5643000	total: 13.4ms	remaining: 211ms
6:	learn: 0.5616973	total: 16.1ms	remaining: 215ms
7:	learn: 0.5599742	total: 18.5ms	remaining: 213ms
8:	learn: 0.5567850	total: 20.8ms	remaining: 210ms
9:	learn: 0.5476675	total: 23.1ms	remaining: 208ms
10:	learn: 0.5444201	total: 25.5ms	remaining: 206ms
11:	learn: 0.5416670	total: 27.3ms	remaining: 200ms
12:	learn: 0.5397606	total:

93:	learn: 0.3854056	total: 200ms	remaining: 12.8ms
94:	learn: 0.3841831	total: 203ms	remaining: 10.7ms
95:	learn: 0.3835289	total: 205ms	remaining: 8.53ms
96:	learn: 0.3819825	total: 207ms	remaining: 6.39ms
97:	learn: 0.3807386	total: 208ms	remaining: 4.25ms
98:	learn: 0.3795686	total: 210ms	remaining: 2.12ms
99:	learn: 0.3785262	total: 212ms	remaining: 0us
0:	learn: 0.6554503	total: 1.47ms	remaining: 146ms
1:	learn: 0.6136713	total: 2.82ms	remaining: 138ms
2:	learn: 0.5836562	total: 4.3ms	remaining: 139ms
3:	learn: 0.5777075	total: 6.14ms	remaining: 147ms
4:	learn: 0.5748866	total: 8.35ms	remaining: 159ms
5:	learn: 0.5734499	total: 11.9ms	remaining: 186ms
6:	learn: 0.5717756	total: 13.7ms	remaining: 181ms
7:	learn: 0.5664473	total: 15.3ms	remaining: 176ms
8:	learn: 0.5604458	total: 17.8ms	remaining: 180ms
9:	learn: 0.5574748	total: 20.4ms	remaining: 184ms
10:	learn: 0.5499961	total: 22.3ms	remaining: 181ms
11:	learn: 0.5454781	total: 25.7ms	remaining: 189ms
12:	learn: 0.5426775	total

0:	learn: 0.6514462	total: 2.22ms	remaining: 220ms
1:	learn: 0.6425571	total: 4.06ms	remaining: 199ms
2:	learn: 0.6117248	total: 5.64ms	remaining: 182ms
3:	learn: 0.5831144	total: 7.26ms	remaining: 174ms
4:	learn: 0.5780461	total: 8.72ms	remaining: 166ms
5:	learn: 0.5732813	total: 10.4ms	remaining: 163ms
6:	learn: 0.5715907	total: 13ms	remaining: 173ms
7:	learn: 0.5627911	total: 15.3ms	remaining: 176ms
8:	learn: 0.5616429	total: 17ms	remaining: 171ms
9:	learn: 0.5526315	total: 18.5ms	remaining: 166ms
10:	learn: 0.5499218	total: 20.5ms	remaining: 166ms
11:	learn: 0.5446703	total: 22.8ms	remaining: 168ms
12:	learn: 0.5421630	total: 25.6ms	remaining: 171ms
13:	learn: 0.5365427	total: 27.7ms	remaining: 170ms
14:	learn: 0.5335487	total: 29.7ms	remaining: 168ms
15:	learn: 0.5308184	total: 31.1ms	remaining: 163ms
16:	learn: 0.5287615	total: 32.6ms	remaining: 159ms
17:	learn: 0.5260882	total: 34.1ms	remaining: 155ms
18:	learn: 0.5219874	total: 35.9ms	remaining: 153ms
19:	learn: 0.5196622	total

60:	learn: 0.4495472	total: 109ms	remaining: 69.8ms
61:	learn: 0.4479378	total: 112ms	remaining: 68.4ms
62:	learn: 0.4471259	total: 114ms	remaining: 67.2ms
63:	learn: 0.4458910	total: 117ms	remaining: 65.6ms
64:	learn: 0.4426536	total: 118ms	remaining: 63.7ms
65:	learn: 0.4405116	total: 120ms	remaining: 61.8ms
66:	learn: 0.4387331	total: 121ms	remaining: 59.8ms
67:	learn: 0.4372186	total: 123ms	remaining: 58ms
68:	learn: 0.4363955	total: 125ms	remaining: 56.1ms
69:	learn: 0.4355905	total: 127ms	remaining: 54.3ms
70:	learn: 0.4316699	total: 129ms	remaining: 52.6ms
71:	learn: 0.4295358	total: 130ms	remaining: 50.7ms
72:	learn: 0.4283162	total: 132ms	remaining: 48.9ms
73:	learn: 0.4271486	total: 134ms	remaining: 47.1ms
74:	learn: 0.4257747	total: 135ms	remaining: 45.1ms
75:	learn: 0.4245082	total: 137ms	remaining: 43.3ms
76:	learn: 0.4229895	total: 139ms	remaining: 41.5ms
77:	learn: 0.4220646	total: 141ms	remaining: 39.9ms
78:	learn: 0.4212159	total: 144ms	remaining: 38.3ms
79:	learn: 0.4

87:	learn: 0.4198990	total: 160ms	remaining: 21.8ms
88:	learn: 0.4189564	total: 162ms	remaining: 20ms
89:	learn: 0.4175417	total: 165ms	remaining: 18.3ms
90:	learn: 0.4167190	total: 167ms	remaining: 16.5ms
91:	learn: 0.4158355	total: 169ms	remaining: 14.7ms
92:	learn: 0.4147889	total: 171ms	remaining: 12.9ms
93:	learn: 0.4141114	total: 174ms	remaining: 11.1ms
94:	learn: 0.4134063	total: 176ms	remaining: 9.24ms
95:	learn: 0.4129354	total: 177ms	remaining: 7.39ms
96:	learn: 0.4123659	total: 179ms	remaining: 5.53ms
97:	learn: 0.4113543	total: 181ms	remaining: 3.69ms
98:	learn: 0.4104368	total: 183ms	remaining: 1.84ms
99:	learn: 0.4097784	total: 184ms	remaining: 0us
0:	learn: 0.6472375	total: 1.84ms	remaining: 182ms
1:	learn: 0.6113585	total: 3.5ms	remaining: 172ms
2:	learn: 0.5883125	total: 4.99ms	remaining: 161ms
3:	learn: 0.5848834	total: 6.63ms	remaining: 159ms
4:	learn: 0.5826578	total: 8.17ms	remaining: 155ms
5:	learn: 0.5802450	total: 10.1ms	remaining: 158ms
6:	learn: 0.5785983	tota

--- Visualisation ---

In [28]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

rfe_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rfe_precision= rfe_precision.sort_values(by= 'Precision Score', ascending=False)
print(rfe_precision)

                                Model  Precision Score
2      Tuned Decision Tree Precision:         0.369066
8   Tuned GradientBoosting Precision:         0.368065
9                      XGB Precision:         0.365688
10               Tuned XGB Precision:         0.365385
4      Tuned Random Forest Precision:         0.365160
7         GradientBoosting Precision:         0.362035
12              Tuned LGBM Precision:         0.362004
14          Tuned CatBoost Precision:         0.360448
3            Random Forest Precision:         0.357364
13                CatBoost Precision:         0.357190
6           Tuned AdaBoost Precision:         0.355453
11                    LGBM Precision:         0.355334
5                 AdaBoost Precision:         0.354573
1            Decision Tree Precision:         0.353575
0      Logistic Regression Precision:         0.348739


In [29]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

rfe_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rfe_accuracy= rfe_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rfe_accuracy)

                               Model  Accuracy Score
0      Logistic Regression Accuracy:        0.477467
1            Decision Tree Accuracy:        0.471192
2      Tuned Decision Tree Accuracy:        0.446663
9                      XGB Accuracy:        0.438106
3            Random Forest Accuracy:        0.431831
12              Tuned LGBM Accuracy:        0.431261
4      Tuned Random Forest Accuracy:        0.430690
14          Tuned CatBoost Accuracy:        0.428408
11                    LGBM Accuracy:        0.426697
5                 AdaBoost Accuracy:        0.420422
8   Tuned GradientBoosting Accuracy:        0.419281
10               Tuned XGB Accuracy:        0.410154
7         GradientBoosting Accuracy:        0.400456
13                CatBoost Accuracy:        0.395893
6           Tuned AdaBoost Accuracy:        0.390759


In [30]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

rfe_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rfe_auc= rfe_auc.sort_values(by= 'AUC Score', ascending=False)
print(rfe_auc)

                          Model  AUC Score
9                      XGB AUC:   0.516425
8   Tuned GradientBoosting AUC:   0.515969
2      Tuned Decision Tree AUC:   0.515491
4      Tuned Random Forest AUC:   0.510773
10               Tuned XGB AUC:   0.506907
3            Random Forest AUC:   0.503930
7         GradientBoosting AUC:   0.503342
5                 AdaBoost AUC:   0.499219
0      Logistic Regression AUC:   0.498130
14          Tuned CatBoost AUC:   0.494810
1            Decision Tree AUC:   0.494087
13                CatBoost AUC:   0.491543
11                    LGBM AUC:   0.488971
12              Tuned LGBM AUC:   0.487560
6           Tuned AdaBoost AUC:   0.469669
